<a href="https://colab.research.google.com/github/shrr98/FP-KK/blob/master/orthopedic_pso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('/content/drive/My Drive/Kuliahku/KK/column_3C_weka.csv')

In [4]:
import numpy as np

arr = df.to_numpy()

arr

array([[63.027817500000005, 22.55258597, 39.60911701, ..., 98.67291675,
        -0.254399986, 'Hernia'],
       [39.05695098, 10.06099147, 25.01537822, ..., 114.4054254,
        4.564258645, 'Hernia'],
       [68.83202098, 22.21848205, 50.09219357, ..., 105.9851355,
        -3.530317314, 'Hernia'],
       ...,
       [61.44659663, 22.694968300000003, 46.17034732, ..., 125.6707246,
        -2.707879517, 'Normal'],
       [45.25279209, 8.693157364, 41.5831264, ..., 118.5458418,
        0.214750167, 'Normal'],
       [33.84164075, 5.073991409, 36.64123294, ..., 123.9452436,
        -0.199249089, 'Normal']], dtype=object)

# Shuffle datasets per class

In [5]:
n_hernia = 60
n_normal = 100
n_spondyl = 150

start = 0
for x in (n_hernia, n_spondyl, n_normal):
  np.random.shuffle(arr[start:start + x])
  start += x

arr

array([[54.12492019, 26.65048856, 35.32974693, ..., 121.447011,
        1.5712048159999998, 'Hernia'],
       [63.027817500000005, 22.55258597, 39.60911701, ..., 98.67291675,
        -0.254399986, 'Hernia'],
       [50.81926781, 15.40221253, 42.52893886, ..., 112.19280400000001,
        10.86956554, 'Hernia'],
       ...,
       [65.7563482, 13.20692644, 43.99999999, ..., 129.3935728,
        -1.9821200380000001, 'Normal'],
       [30.74193812, 13.35496594, 35.90352597, ..., 142.4101072,
        -2.005372903, 'Normal'],
       [42.51561014, 16.54121618, 41.99999999, ..., 120.631941,
        7.876730692000001, 'Normal']], dtype=object)

In [6]:
val_split = 0.2
test_split = 0.2

val_set = None
train_set = None
test_set = None
processed = 0

for x in (n_hernia, n_spondyl, n_normal ):
  n_val = int(val_split * x)
  n_test = int(test_split * x)
  if processed==0:
    val_set = np.array(arr[processed:processed + n_val, :])
    processed += n_val
    test_set = np.array(arr[processed:processed + n_test, :])
    train_set = np.array(arr[processed:processed + x - n_val - n_test, :])
    processed += x - n_val - n_test
    continue
  val_set = np.concatenate((val_set, arr[processed:processed + n_val, :]))
  processed += n_val
  test_set = np.concatenate((test_set, arr[processed:processed + n_test, :]))
  train_set = np.concatenate((train_set, arr[processed : processed + (x - n_val - n_test), :]))
  processed += x - n_val - n_test
  print(val_set.shape)

print(f'Train : {train_set.shape}')
print(f'Val : {val_set.shape}')
print(f'Test : {test_set.shape}')

(42, 7)
(62, 7)
Train : (186, 7)
Val : (62, 7)
Test : (62, 7)


In [7]:
x_train = np.array(train_set[:,:6], dtype=np.float32)
y_train = train_set[:, 6]

x_val = np.array(val_set[:,:6], dtype=np.float32)
y_val = val_set[:,6]

x_test = np.array(test_set[:,:6], dtype=np.float32)
y_test = test_set[:,6]

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)


(186, 6) (186,)
(62, 6) (62,)
(62, 6) (62,)


In [0]:
for i in range(x_train.shape[1]):
  mini = np.min(x_train[:,i])
  maxi = np.max(x_train[:,i])
  mean = np.mean(x_train[:,i])
  x_train[:,i] = np.subtract(x_train[:,i], mean) / float(maxi-mini)
  
  mini = np.min(x_val[:,i])
  maxi = np.max(x_val[:,i])
  mean = np.mean(x_val[:,i])
  x_val[:,i] = np.subtract(x_val[:,i], mean) / float(maxi-mini)

  mini = np.min(x_test[:,i])
  maxi = np.max(x_test[:,i])
  mean = np.mean(x_test[:,i])
  x_test[:,i] = np.subtract(x_test[:,i], mean) / float(maxi-mini)

In [9]:
!pip install pyswarms
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [0]:
classes = np.unique(train_set[:,6])

y_val = np.array([np.where(classes==x)[0][0] for x in y_val])

y_train = np.array([np.where(classes==x)[0][0] for x in y_train])

y_test = np.array([np.where(classes == x)[0][0] for x in y_test])

In [0]:
    # Neural network architecture
    n_inputs = 6
    n_hidden = 20
    n_classes = 3


In [0]:
pos_best_val = np.zeros((n_inputs*n_hidden + n_hidden + n_hidden*n_classes + n_classes), dtype=np.float32)
acc_best_val = 0

In [0]:
 # Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Roll-back the weights and biases
    start = 0
    W1 = params[0:n_inputs * n_hidden].reshape((n_inputs,n_hidden))
    start += n_inputs * n_hidden
    b1 = params[start:start + n_hidden].reshape((n_hidden,))
    start += n_hidden
    W2 = params[start:start + n_hidden * n_classes].reshape((n_hidden,n_classes))
    start += n_hidden * n_classes
    b2 = params[start:start + n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = x_train.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = x_train.shape[0] # Number of samples
    corect_logprobs = -np.log(probs[range(N), y_train])
    loss = np.sum(corect_logprobs) / N

    global pos_best_val
    global acc_best_val
    acc = (predict(x_val, params) == y_val).mean()
    if acc > acc_best_val:
      acc_best_val = acc
      pos_best_val = np.array(params)

    return loss

In [0]:
def predict(X, params):

    # Roll-back the weights and biases
    start = 0
    W1 = params[0:n_inputs * n_hidden].reshape((n_inputs,n_hidden))
    start += n_inputs * n_hidden
    b1 = params[start:start + n_hidden].reshape((n_hidden,))
    start += n_hidden
    W2 = params[start:start + n_hidden * n_classes].reshape((n_hidden,n_classes))
    start += n_hidden * n_classes
    b2 = params[start:start + n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [0]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [16]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * 20) + (20 * 3) + 20 + 3
optimizer = ps.single.GlobalBestPSO(n_particles=200, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2019-12-09 00:09:42,472 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.162
2019-12-09 00:10:43,543 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1619051332768464, best pos: [-5.46203384e-01 -1.01964247e-01 -6.93633546e-01  4.60834176e-01
 -7.39998204e-01  1.81605358e-01 -1.38306168e+00 -1.02044583e+00
  1.74447566e+00  1.68378792e+00 -1.77998512e+03  2.34208449e+00
  8.86757625e-01  9.34693134e-01  2.96943051e+00 -1.61644160e-01
  1.21301581e+00  2.15521964e+00 -2.33850392e-01  2.59490447e+00
 -3.41537855e-01 -1.55950819e+00  4.03426372e+00  1.30253903e+00
  6.20517286e-01  4.33085873e+00  1.66364162e+00 -3.60178646e-01
 -2.29560450e+00 -7.34874827e-01 -2.54520998e+00  8.10820447e+00
  1.64634716e+00  1.54965308e+00  6.74065655e-01 -1.74163828e-01
  1.07596407e+00 -1.73844401e+00  9.01358590e-01  1.76997186e+00
  1.57057236e-01 -3.55774

# Accuracies on best position from training set only

In [17]:
print(f'Accuracy on Training set: {(predict(x_train, pos) == y_train).mean()}')

Accuracy on Training set: 0.9247311827956989


In [18]:
print(f'Accuracy on Validation set: {(predict(x_val, pos) == y_val).mean()}')

Accuracy on Validation set: 0.8387096774193549


In [19]:
print(f'Accuracy on Test set: {(predict(x_test, pos) == y_test).mean()}')

Accuracy on Test set: 0.8064516129032258


# Accuracies on best position on maximum validation accuracy

In [20]:
print(f'Accuracy on Training set: {(predict(x_train, pos_best_val) == y_train).mean()}')

Accuracy on Training set: 0.6720430107526881


In [21]:
print(f'Accuracy on Validation set: {(predict(x_val, pos_best_val) == y_val).mean()}')

Accuracy on Validation set: 1.0


In [22]:
print(f'Accuracy on Test set: {(predict(x_test, pos_best_val) == y_test).mean()}')


Accuracy on Test set: 0.9193548387096774


# Save weights

In [0]:
import codecs, json

file_path = "/content/drive/My Drive/Kuliahku/KK/weights1_bestpos.json"
json.dump(pos.tolist(), codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4) ### this saves the array in .json format